<a href="https://colab.research.google.com/github/laiadc/DL-schrodinger/blob/main/Random_potentials1D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/laiadc/DL-schrodinger.git

Cloning into 'DL-schrodinger'...
remote: Enumerating objects: 33, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 101 (delta 7), reused 28 (delta 4), pack-reused 68
Receiving objects: 100% (101/101), 288.60 MiB | 27.70 MiB/s, done.
Resolving deltas: 100% (9/9), done.
Checking out files: 100% (76/76), done.


In [2]:
cd DL-schrodinger

/content/DL-schrodinger


In [4]:
%tensorflow_version 2.x

# Integrating the Schrödinger equation with Deep Learning


In this notebook, we are going to look for the solutions of the Schrödinger equation for multiple systems. In order to do so, we will use Deep Learning models for each system.

We will begin by finding the stationary states of the Time-Independent Schrödinger equation:

$$
\Big( - \frac{\hbar^2}{2m} \frac{\partial^2}{\partial x^2} + V(x) \Big) \phi_n(x) = E_n \phi_n(x) 
$$

Where $\hbar$ is the reduced Plank constant, $m$ is the mass of the particle, $V(x)$ is the potential under which the particle evolves. $\phi_n(x)$ is the $n$-th stationary state of the quantum system, with energy $E_n$. The subscrit $n$ can be discrete ($n \in \mathbb{Z}$) or continuous ($n \in \mathbb{R}$).

The stationary wave functions $\phi_n(x)$ have a trivial time evolution:

$$
\phi_n(x,t) = \phi_n(x) e^{-i \frac{E_n}{\hbar}t}
$$

They form a basis of the Hilbert Space of the Hamiltonian of the system

$$
\hat{H} = \frac{\hat{p}^2}{2m} + V(\hat{x})
$$

Therefore, the evolution of an arbitrary state of the system $\psi(x,t)$ will be given by:

$$
\psi(x,t) = \sum_k c_k \phi_k(x) e^{-i \frac{E_k}{\hbar}t} 
$$

Where $c_k$ are linear coefficients and the sum over $k$ can be either discrete or continuous (an integral). The modulus square of the coefficients $|c_k|^2$ represents the probability of being in the $k$-th excited state. 



# Variational method with the H.O basis

Given a Hamiltonian

$$
\hat{H} = \frac{\hat{p}^2}{2m} + V(\hat{x}),
$$

the mean energy of a wavefunction $\psi(x)$ is

$$
<H> = <\psi|\hat{H}|\psi> = \int_{-\infty}^\infty \psi^*(x) H(x) \psi(x)dx
$$

For simplicity, from now on we will choose $m=1$ and $\hbar=1$ and thus omit these terms in the calculations. 

### Variational principle
The variational principle states that the mean energy (under a Hamiltonian $\hat{H}$) of a wavefunction is greater or equal to the ground state energy of such Hamiltonian. That is:

$$
E_0 \leq <H> = <\psi|\hat{H}|\psi> \quad \forall |\psi> \in \mathcal{H}
$$

This principle can be extended to higher eigen energies by imposing that the state $\psi$ is orthogonal to the previous eigenstates

### The H.O basis

We choose as a basis of $\mathcal{H}$ the eigenfunctions of the Harmonic Oscillator with $m=1$, $\hbar=1$ and $\omega=1$. 

$$
\phi_n(x) = \frac{1}{\sqrt{n!2^n \sqrt{\pi}}} e^{-x^2/2} H_n(x)
$$

Where $H_n(x)$ is the $n$th Hermite polynomial, which is defined by the recurrence:

$$
H_n(x) = 2xH_{n-1}(x) - 2n H_{n-2}(x)\\
H_0(x) = 1
$$

Since $\{\phi_n(x)\}_n$ form a basis of $\mathcal{H}$, we can write any wavefunction $\psi(x)$ as a linear combination of the eigenfunctions:

$$
\psi(x) = \sum_{n=0}^\infty a_n \phi_n(x)
$$

Thus, the mean energy of $\psi$ is

$$
<H> = <\psi|\hat{H}|\psi> = \int_{-\infty}^\infty \Big(\sum_{n=0}^\infty a_n \phi_n(x)\Big) \hat{H} \Big(\sum_{m=0}^\infty a_m \phi_m(x)\Big) dx =\\
\sum_{n=0}^\infty \sum_{m=0}^\infty a_n a_m \int_{-\infty}^\infty \phi_n(x) H(x) \phi_m(x)dx = \sum_{n=0}^\infty \sum_{m=0}^\infty a_n a_m C_{nm}
$$

where

$$
C_{nm} = \int_{-\infty}^\infty A_n e^{-x^2/2} H_n(x) \Big(-\frac{1}{2} \frac{\partial^2}{\partial x^2} + V(x) \Big) A_m e^{-x^2/2} H_m(x) dx, \quad A_n = \frac{1}{\sqrt{n!2^n \sqrt{\pi}}}
$$

In order to find a good estimation of the ground energy, we will generate a finite basis of H.O eigenstates $\{\phi_n\}_{n=0}^N$ and then find the coefficients $\{a_n\}$ which minimize the mean energy $<H>$. 

## Finding the coefficients $\{a_n\}$

In order to find the coefficients $\{a_n\}$ which minimize the energy $<H>$ we use the Lagrange Multipliers Theorem, which state that the local minimum of a function $F$, under a constraing $G$ is the solution of

$$
\nabla F = \lambda \nabla G, \quad \lambda \in \mathbb{R}
$$

In this case $F$ is the mean energy

$$
F(\{a_n\}) = <H>
$$

and $G$ is the normalization constraint. Since $\{\phi_n\}$ is a basis of $\mathcal{H}$, then

$$
G(\{a_n\}) = \sum_{n=0}^N a_n^2 = 1
$$

Now we have to calculate the partial derivatives of $F$ and $G$.

$$
\frac{\partial F}{\partial a_i} = \frac{\partial}{\partial a_i} \Big( \sum_{n=0}^N \sum_{m=0}^N a_n a_m C_{nm} \Big)=  \frac{\partial}{\partial a_i} \Big( \sum_{n=0} a_n \Big) \Big(\sum_{m=0}^N a_m C_{nm}\Big) + \frac{\partial}{\partial a_i} \Big( \sum_{m=0} a_m \Big) \Big(\sum_{n=0}^N a_n C_{nm}\Big) =\\
\sum_{m=0}^N a_m C_{im} + \sum_{n=0}^N a_n C_{ni} = \sum_{n=0}^N a_n (C_{in} + C_{ni})
$$

Thus, the gradient of $F$ is a linear system:

$$
\nabla F(\vec{a}) = D \vec{a}, \quad \vec{a} = 
\begin{pmatrix} 
a_1\\
\vdots\\
a_N
\end{pmatrix}, \quad D \in \mathcal{M}_{N}(\mathbb{R}), \ [D]_{ij} = C_{ij} + C_{ji}
$$

Then,

$$
\frac{\partial G}{\partial a_i} = \frac{\partial }{\partial a_i} \Big(\sum_{n=0}^N a_n^2\Big) = 2a_i
$$

Therefore, the Lagrange Multiplier equation becomes

$$
\nabla F(\vec{a}) = \lambda \nabla G(\vec{a}) \Longleftrightarrow D \vec{a} = 2\lambda \vec{a}
$$

Which is an eigenvalue problem. The solution will then be found by solving the eigenvalue problem and then selecting the vector $\vec{a_0}$ which minimizes $<H>$. Since the basis $\phi_n(x)$ is finite (we take up to $N$ functions), the solution will be an approximation of the true eigenvector. When $N \rightarrow \infty$ the solution $\psi(x)$ will converge to the ground state of $H$. Moreover, since the eigenvectors of $D$ are orthonormal, the vector with the $n$th lowest energy will be an approximation to the $n$th excited state of the Hamiltonian.  


## Integrating Hermite Polynomials

In order to generate a basis of the Hilbert Space that we will use to approximate the gound state wavefunctions, we need to perform some integrals involving Hermite Polynomials:

$$
I(n,m,r) = \int_{-\infty}^\infty x^r e^{-x^2} H_n(x) H_m(x) dx
$$

Where $H_n(x)$ is the $n$-th Hermite polynomial, defined by the recurrence:

$$
H_n(x) = \frac{1}{2x}\Big(H_{n+1}(x) + 2nH_{n-1}(x)\Big)\\
H_0(x) = 1
$$

Because of the normalization constraints of the Harmonic Oscillator, we know that:

$$
I(n,m,0) = \sqrt{\pi} 2^n n! \delta_{n,m} 
$$

Therefore,

$$
I(n,m,r) = \int_{-\infty}^\infty x^r e^{-x^2} H_n(x) H_m(x) dx = \int_{-\infty}^\infty x^r e^{-x^2}\frac{1}{2x}\Big(H_{n+1}(x) + 2nH_{n-1}(x)\Big)H_m(x)dx = \\
\frac{1}{2}I(n+1,m,r-1) + nI(n-1,m,r-1)
$$

Using this recurrence, we will calculate the values of $I(n,m,r)$.

## Calculating $C_{nm}$

In order to diagonalize the matrix $D$, we need to calculate the coefficients $C_{nm}$

$$
C_{nm} = A_nA_m \int_{-\infty}^\infty e^{-x^2/2} H_n(x) (-\frac{1}{2} \frac{\partial^2}{\partial x^2} + V(x)) H_m(x) e^{-x^2/2} dx
$$
$$
A_n = \frac{1}{\sqrt{n! 2^n \sqrt{\pi}}}
$$

In order to do so we need to calculate:

$$
\frac{\partial^2}{\partial x^2}(H_m(x) e^{-x^2/2} ) = e^{-x^2/2}\Big((x^2-1) H_m(x) - 4mx H_{m-1}(x) + 4m(m-1)H_{m-2}(x)\Big) := e^{-x^2/2} P(x)
$$

$$
C_{nm} = A_n A_m \Big( - \frac{1}{2} \int_{-\infty}^\infty  H_n(x) P(x) e^{-x^2} dx + \int_{-\infty}^\infty e^{-x^2} H_n(x)H_m(x)V(x) dx\Big) = \\
A_nA_m\Big(- \frac{1}{2} I(n,m,2) + 1/2 I(n,m,0) + 2mI(n,m-1,1) - 2m(m-1)I(n, m-2, 0) + I_V\Big)
$$

Where $I_V$ is the integral corresponding to the potential $V(x)$. If

$$
V(x) = \sum_{i=1}^N \alpha_i x^i
$$

Then

$$
I_V = \sum_{i=1}^N \alpha_i I(n,m,i)
$$




## Code

In the following cell we define a class which calculates the ground state energy and wavefunction of a potential of the form:

$$
V(x) = \sum_{i=1}^k \alpha_i x^i
$$

The calculation is analytical (by using the Hermite integrals). This class generates random values of $\vec{\alpha}$ following a uniform distribution between $\vec{\alpha}_{min}$ and $\vec{\alpha}_{max}$. Then it calculates the $n_{state}$-th excited state for these potentials. It returns the energy of such states, the coefficients $\{a_n\}$, the potentials $V(x)$ and the wavefunctions $\psi(x) = \sum_{n=0}^N a_n \phi_n(x)$. 

In [5]:
#@title Class to find eigenstates using the H.O basis
#@markdown Double click to see the code
import numpy as np
from scipy.special import factorial
from scipy import linalg as LA
import scipy.sparse as sps
from scipy.linalg import eigh
from scipy.special import eval_hermite
from scipy.signal import argrelextrema

class eigen_state_potential:

  def __init__(self, alpha_min=None, alpha_max=None, N=10):
    '''
    Class to generate data (V(x) and phi(x) ground state) for potentials of the form
    V(x) = sum_i alpha_i x^i, using the H.O basis
    Args:
      alpha_min: vector of length N, with the minimum value of the coefficients alpha
      alpha_max: vector of length N, with the maximum value of the coefficients alpha
      the values of alpha will be randomly distributed in [alpha_min, alpha_max]
    '''
    if len(alpha_min)!=len(alpha_max):
      print("Error. Inconsisten shapes")
    self.alpha_min = np.array(alpha_min)
    self.alpha_max = np.array(alpha_max)
    self.N = N # Length of H.O basis
    self.k = len(alpha_min) #Number of alphas for V(x)

  def I_nmr(self,n,m,r):
    '''
    Calculates the value of the integral of the Hermitte polynomials
    Args:
      n (int): n of I(n,m,r)
      m (int): m of I(n,m,r)
      r (int): r of I(n,m,r)
    Returns:
      I(n,m,r)
    '''
    if r<0 or n<0 or m<0:
      return 0
    if r==0:
      if n==m:
        return np.sqrt(np.pi)*2**n*factorial(n)
      else:
        return 0
    return 1./2*self.I_nmr(n+1,m,r-1) + n*self.I_nmr(n-1,m,r-1)

  def C_nm(self,n,m, alphas):
    '''
    Calculates the coefficient C_{nm} for the potential V(x) = \sum_i alpha[i]x^i
    Args:
      n (int): n of C_nm
      m (int): m of C_nm
      alphas (np.array): size k. Coefficients of the potential V(x)
    Returns:
      C_{nm}
    '''
    An = 1./np.sqrt(np.sqrt(np.pi)*factorial(n)*2**n)
    Am = 1./np.sqrt(np.sqrt(np.pi)*factorial(m)*2**m)
    I1 = -1/2*self.I_nmr(n,m,2)
    I2 = 1/2*self.I_nmr(n,m,0)
    I3 = 2*m*self.I_nmr(n,m-1,1)
    I4 = -2*m*(m-1)*self.I_nmr(n, m-2,0)
    Iv = 0
    for i in range(len(alphas)):
      Iv+=alphas[i]*self.I_nmr(n,m,i)
    return An*Am*(I1+I2+I3+I4+Iv)

  def find_eigen_state(self,alphas, n_state=0):
    '''
    Finds the eigen state of a potential V(x) = sum_i alpha_i x^i
    Args:
      alphas(np array): size k. Coefficients of the potential V(x)
      n_state (int): Number of excited state (default n_state=0, ground state)
    Returns:
      E_a (float): Energy of the ground state for potential V
      a (np.array): size N. Coefficients in the basis of the H.O potential
    '''
    N = self.N
    # 0. Generate matrix of C_nm
    C = np.zeros((N,N))
    for n in range(N):
      for m in range(N):
        C[n,m] = self.C_nm(n,m,alphas)

    # 1. Generate matrix D
    D = np.zeros((N,N))
    for n in range(N):
      for m in range(n+1):
        D[n,m] = C[n,m] + C[m,n]
        D[m,n] = D[n,m]

    # 2. Diagonalize matrix D

    vaps, veps = eigh(D)

    # 3. Calculate <H> for all a

    Hs = np.zeros(N)
    for i in range(N):
      a = veps[:, i]
      for n in range(N):
        for m in range(N):
          Hs[i]+=a[n]*a[m]*C[n,m]

    # 4. We choose the vector which minimizes <H>
    # If n_state!=0, we choose the vector with n_state-th lowest energy
    # as an approximation of the n_state excited state 
    sel = np.argsort(Hs)[n_state]#np.argmin(Hs)
    a = veps[:, sel] # Final value of eigenvalues for state n_state
    E_a = Hs[sel] # Value of the energy
    return E_a, a

  def generate_data(self, n_samples, alpha=np.array([None]), n_state=0, display=100):
    '''
    Generates samples of potentials  with random coefficients and finds the n_state excited state for them
    Args:
      n_samples (int): Number of samples of potentials (alphas)
      alpha (np.array): Values of alpha. If you want to generate them randomly, don't provide anything
      n_state (int): Number of excited state (default n_state=0, ground state)
    Returns:
      E (np.array): size n_samples. Ground energy for each V
      a (np.array): size n_samples x N. Coefficients in the H.O basis for each V
      alpha (np.array): size n_samples x k. Coefficients of the potentials V(x)
    '''
    data = np.zeros((n_samples, self.N))

    # Generate random value of alphas
    if (alpha==None).any():
      print("Random alphas")
      r_alpha = np.random.random((n_samples, self.k)) # Values between 0 and 1
      alpha = r_alpha*(self.alpha_max - self.alpha_min)+ self.alpha_min # random alpha
    
    # Prepare vectors of energies and coefficients
    E = np.zeros(n_samples)
    a = np.zeros((n_samples, self.N))
    # Find ground state for each sample
    for i in range(n_samples):
      E_new, a_new = self.find_eigen_state(alpha[i,:], n_state)
      if i%display==0:
        print("\rGenerating data: {}/{}".format(i,n_samples), end='')
      E[i] = E_new
      a[i,:] = a_new
    return E, a, alpha     

  def evaluate_potential(self, xmin, xmax, n_points, alpha):
    '''
    Given the coeefficients alphas, it evaluates the potential in V(x)
    Args:
      xmin(float): minimum value of x
      xmax (float): maximum value of x
      n_points (int): Number of points between xmin and xmax
      alpha (np.array): size n_samples x k. Matrix of coefficients of V(x) (each row a different potential)
    Returns:
      V(np.array): size n_samples x n_points. V(x) for every sample
      x(np.array): size n_points. Values of x
    '''
    x = np.arange(xmin, xmax, (xmax - xmin)/n_points)
    n_samples, k = alpha.shape
    V = np.zeros((n_samples, n_points))
    x_mat = (x**np.arange(k)[:,None])# Matrix of powers of x: x^0, x^1, x^2, ..., x^N (in every row)
    V = np.zeros((n_samples, n_points))# V(x) in each row different alpha
    for i in range(n_samples):
      for j in range(n_points):
        V[i,j] = np.dot(alpha[i,:],x_mat[:,j])
    
    return V, x

  def HO_wavefunction(self, n, xmin, xmax, n_points):
    '''
    Returns the nth eigenfunction of the harmonic oscillator in the points x
    Args:
      n (int): Energy level
      xmin(float): minimum value of x
      xmax (float): maximum value of x
      n_points (int): Number of points between xmin and xmax
    Returns:
      phi_n (np.array): size n_points. Phi_n(x)
    '''
    x = np.arange(xmin, xmax, (xmax - xmin)/n_points)
    herm = eval_hermite(n, x) # H_n(x)
    exp = np.exp(- x**2/2) # Exponential term
    phi_n = exp*herm
    
    # Normalization
    h = (xmax - xmin)/n_points
    C = 1./np.sqrt(np.sum(phi_n*phi_n*h))
    phi_n = C*phi_n

    return phi_n

  def final_wavefunction(self, xmin, xmax, n_points, a):
    '''
    Returns the final wavefunctions psi(x) = sum_i alpha_i phi_i(x) for each alpha.
    Args:
      xmin(float): minimum value of x
      xmax (float): maximum value of x
      n_points (int): Number of points between xmin and xmax
      a (np.array): size n_samples x N. Coefficients in the H.O basis for each V
    Returns:
      waves(np.array): size n_samples x n_points. psi(x) for each value of V (given by alpha)
    '''
    x = np.arange(xmin, xmax, (xmax - xmin)/n_points)
    n_samples, _ = a.shape
    # Construct matrix of phi_n
    phis = np.zeros((self.N, n_points))
    for i in range(self.N):
      phis[i,:] = self.HO_wavefunction(i, xmin, xmax, n_points)
    
    waves = np.zeros((n_samples, n_points))
    for i in range(n_samples):
      for j in range(n_points):
        waves[i,j] = np.dot(a[i,:],phis[:,j])
      # convention: To choose the phase we make the maximums be first
      w = waves[i,:]
      maxi = argrelextrema(w, np.greater)[0]
      mini = argrelextrema(w, np.less)[0]
      idx2= np.abs(w[maxi])>5e-2
      maxi = maxi[idx2]
      idx2= np.abs(w[mini])>5e-2
      mini = mini[idx2]
      if len(maxi)==0 and len(mini)>0:
        waves[i,:] = -waves[i,:]
      elif len(mini)>0 and len(maxi)>0 and mini[0]<maxi[0]:
        waves[i,:] = -waves[i,:]

    return waves, x, phis

# Ground state potentials

Now that we have generated the code to approximate the eigenstates of a random potential, we are going to generate random potentials of the form:

$$
V(x) = \alpha_0 + \alpha_1 x + \alpha_2 x^2 + \alpha_3 x^3 + \alpha_4 x^4
$$

To ensure that the eigenstates have discrete energies (and thus are physical states), we will impose some properties on the coefficients, so that the even terms ($x^2$ and $x^4$) dominate over the odd term ($x^3$). Also, we allow the potential to be negative and non-centered by including negative values of $\alpha_0$ and $\alpha_1$. Finally, we will use small values of the coefficients so that the potential does not achieve very high values, which can lead to numerical unstability.


In [ ]:
#@title Generate data

#@markdown Double click to see the code


#@markdown ---
#@markdown ### Enter parameters:
xmin = -8#@param {type:"number"}
xmax = 8#@param {type:"number"}
N=15#@param {type:"slider", min:5, max:30, step:1}
n_state=0#@param {type:"slider", min:0, max:10, step:1}
n_points = 200#@param {type:"integer"}
n_samples=100#@param {type:"integer"}
#@markdown ---

alpha_min = np.array([-150,-10,0.3,-0.1,0])/50
alpha_max = np.array([50,5,1.0,0.1,0.25])/50

k = alpha_min.shape[0]

r_alpha = np.random.random((int(n_samples*0.8), k)) # Values between 0 and 1
alpha1 = r_alpha*(alpha_max - alpha_min)+ alpha_min # random alpha

alpha_min2 = np.array([-10,-1,0.3,0,0])
alpha_max2 = np.array([5,1,1.0,0,0])

r_alpha = np.random.random((int(n_samples*0.2), k)) # Values between 0 and 1
alpha2 = r_alpha*(alpha_max2 - alpha_min2)+ alpha_min2 # random alpha
alpha = np.concatenate((alpha1, alpha2))


## Convergence of the method

In order to choose the value of $N$, we will generate the ground-state energies for different vales of $N$ and compute the difference. When the value of the energy stabilizes up to certain precision $\gamma$, we will fix the value of $N$. We will evaluate the performance for 500 different samples.

**Note: The following cells may take some hours to run**

In [ ]:
gamma = 0.001
Ns = np.arange(3,31)
Es = np.zeros(( Ns.shape[0], n_samples))

for i in range(Ns.shape[0]):
  data_gen = eigen_state_potential(alpha_min, alpha_max, Ns[i])
  E, a, _ = data_gen.generate_data(n_samples, alpha)
  Es[i,:] = E

Generating data: 0/100

In [ ]:
#@title Energies as a function of N
import plotly
from plotly.graph_objs import graph_objs as go
from plotly.subplots import make_subplots
from plotly.offline import iplot


fig = go.Figure()
fig.add_trace(go.Scatter(
    x=Ns,
    y=Es[:,1],
    mode="lines",
    name = "Energy evolution")
    )

fig.add_trace(go.Scatter(
    x=Ns,
    y=np.repeat(Es[-1,1], Ns.shape[0]),
    mode="lines",
    name = "final energy")
    )

fig.update_layout(
    title="Energies as a function of N",
    xaxis_title="N")

fig.show()

We see that the energy stabilizes fast. Now, for $\gamma =0.001$, let's see which is the $N$ needed on average to ensure convergence.

In [ ]:
#@title Evolution of difference of energy
E_dif = np.abs(Es[:-1,:] - Es[1:,:])

E_dif_mean = np.mean(E_dif, axis=1)

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=Ns[1:],
    y=E_dif_mean,
    mode="lines",
    name = "Difference in energy")
    )
fig.add_trace(go.Scatter(
    x=Ns[:-1:],
    y=np.repeat(gamma, (Ns[1:]).shape[0]),
    mode="lines",
    name = "Gamma")
    )

We see that for $N=15$ the difference in the ground energy is below $\gamma$. Thus, we select this value to generate the wavefunctions. 

## Generating the data

In [73]:
#@markdown Double click to see the code \\
#@markdown **Note: This cell may take some time to run**

#@markdown ---
#@markdown ### Enter parameters:
xmin = -8#@param {type:"number"}
xmax = 8#@param {type:"number"}
N=15#@param {type:"slider", min:5, max:30, step:1}
n_state=0#@param {type:"slider", min:0, max:10, step:1}
n_points = 200#@param {type:"integer"}
n_samples=5000#@param {type:"integer"}
#@markdown ---

# We generate the values of alpha
alpha_min = np.array([-150,-10,0.3,-0.1,0])/50
alpha_max = np.array([50,5,1.0,0.1,0.25])/50

k = alpha_min.shape[0]

r_alpha = np.random.random((int(n_samples*0.8), k)) # Values between 0 and 1
alpha1 = r_alpha*(alpha_max - alpha_min)+ alpha_min # random alpha

alpha_min2 = np.array([-10,-1,0.1,0,0])
alpha_max2 = np.array([5,1,1.0,0,0])

r_alpha = np.random.random((int(n_samples*0.2), k)) # Values between 0 and 1
alpha2 = r_alpha*(alpha_max2 - alpha_min2)+ alpha_min2 # random alpha
alpha = np.concatenate((alpha1, alpha2))

data_gen = eigen_state_potential(alpha_min, alpha_max, N)

# Generate the energies, wavefunctions and potentials
E, a, alpha = data_gen.generate_data(n_samples, alpha)
waves, x,phis = data_gen.final_wavefunction( xmin, xmax, n_points, a)
V, _ = data_gen.evaluate_potential( xmin, xmax, n_points, alpha)
idx=-1

Generating data: 4900/5000

In [74]:
#@title Potential and wavefunction
import plotly
from plotly.graph_objs import graph_objs as go
from plotly.subplots import make_subplots
from plotly.offline import iplot

idx +=1

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(
    x=x,
    y=waves[idx,:],
    mode="lines",
    name = "solution")
    )

fig.add_trace(go.Scatter(
    x=x,
    y=V[idx,:],
    mode="lines",
    name = "potential"),
    secondary_y=True
    )

fig.update_layout(
    title="Approximate wavefunctions",
    xaxis_title="x")

fig.show()

## Training a neural network



In [75]:
#@title Class to define the neural network
#@markdown Double click to see the code
import tensorflow as tf
from tensorflow.keras import Model, layers

class FC_Model(tf.keras.Model):
    '''
    Subclassed keras tf.keras.Model API. The input will be the potential V(x)
    and the output will be the wave function phi_n(x).
    Args:
      input_size (int): Number of x points
    Attributes:
      input_size (int): Number of x points
      fc1 (layer): First  fully cinnected layer with 512 filters and relu activation function
      dropout1 (layer): Dropout layer with dropout parameter of 0.2
      fc2 (layer): Second  fully cinnected layer with 256 filters and relu activation function
      dropout2 (layer): Dropout layer with dropout parameter of 0.2
      fc3 (layer): Third  fully cinnected layer with 256 filters and relu activation function
      dropout3 (layer): Dropout layer with dropout parameter of 0.2
      fc4 (layer): Fourth  fully cinnected layer with 128 filters and relu activation function
      dropout4 (layer): Dropout layer with dropout parameter of 0.2
      out (layer): Output layer predicting phi_n(x)
    '''
    def __init__(self,
                 name='fc_model', input_size=100,
                 n1 = 256, n2= 256, n3=128, n4=128, drop=0.1,
                 **kwargs):
        self.input_size = input_size
        super(FC_Model, self).__init__(name=name, **kwargs)

        # Fully connected layer.
        self.fc1 = tf.keras.layers.Dense(n1,  activation='relu')
        # Apply Dropout (if is_training is False, dropout is not applied).
        self.dropout1 = tf.keras.layers.Dropout(rate=drop)

        # Fully connected layer.
        self.fc2 = tf.keras.layers.Dense(n2,  activation='relu')
        # Apply Dropout (if is_training is False, dropout is not applied).
        self.dropout2 = tf.keras.layers.Dropout(rate=drop)

        # Fully connected layer.
        self.fc3 = tf.keras.layers.Dense(n3, activation='relu')
        # Apply Dropout (if is_training is False, dropout is not applied).
        self.dropout3 = tf.keras.layers.Dropout(rate=drop)

        # Fully connected layer.
        self.fc4 = tf.keras.layers.Dense(n4, activation='relu')
        # Apply Dropout (if is_training is False, dropout is not applied).
        self.dropout4 = tf.keras.layers.Dropout(rate=drop)

        # Output layer (fully connected with input_size neurons and linear activation function )
        self.out = tf.keras.layers.Dense(self.input_size, activation ='linear')


    @tf.function
    def call(self, inputs, is_training=False):
        '''
        Forward pass of the fully connected model

        Args:
          inputs (tensor): X data to pass through the network (V(x))
          is_training (bool): If training, True, otherwise, False
        
        Returns:
          out (tensor): Output tensor containing the values of phi_n(x)
        '''
        x = tf.reshape(inputs, tf.constant([-1, self.input_size]))
        x = self.fc1(x)
        x = self.dropout1(x, training=is_training)
        x = self.fc2(x)
        x = self.dropout2(x, training=is_training)
        x = self.fc3(x)
        x = self.dropout3(x, training=is_training)
        x = self.fc4(x)
        x = self.dropout4(x, training=is_training)
        out = self.out(x)
        return out


In [76]:
#@title Class to train the network
#@markdown Double click to see the code
class Training():
  '''
  Performs the training of the autoencoder model using mean absolute error loss

  Args:
    net (Model): Model to train
    learning_rate (float): Learning Rate for Adam optimizer
    training_iters (int): Numer of training iterations
    batch_size (int): Batch size
    display_step (int): Number of iterations to wait to print the current performance of the model
    early_stopping (int): Number of epochs to wait for the validation loss to increase before performing early stopping
    filepath (str): File path to store and recover the model weights
    restore (bool): If true, it looks for existing weights to reestore them

  Attributes: 
    net (Model): Model to train
    learning_rate (float): Learning Rate for Adam optimizer
    training_iters (int): Numer of training iterations
    batch_size (int): Batch size
    display_step (int): Number of iterations to wait to print the current performance of the model
    stopping_step (int): How many epochs we have waited so far without the validation loss decreasing
    early_stopping (int): Number of epochs to wait for the validation loss to increase before performing early stopping
    filepath (str): File path to store and recover the model weights
    restore (bool): If true, it looks for existing weights to reestore them
    loss (function): Loss function to optimize. In this case, mean square error
    optimizer (tf.Optimizer): Adam optimizer for the learning steps
    ckpt (tf.Checkpoint): Checkpoint that stores weights and optimizer state
    manager (tf.CheckpointManager): Controls that not too many checkpoint files are stored 
    

  '''

  def __init__(self, net, learning_rate, training_iters, batch_size,
               display_step, early_stopping=50, filepath=None, restore =True):
    self.net = net
    self.learning_rate = learning_rate
    self.training_iters = training_iters
    self.batch_size = batch_size
    self.display_step = display_step
    self.stopping_step=0
    self.loss = tf.keras.losses.MeanSquaredError()
    self.early_stopping = early_stopping
    self.optimizer = tf.keras.optimizers.Adam(self.learning_rate)
    self.filepath = filepath
    self.ckpt = tf.train.Checkpoint(optimizer=self.optimizer, net=self.net)
    self.manager = tf.train.CheckpointManager(self.ckpt, directory = filepath , max_to_keep=3)
    if restore:
      self.ckpt.restore(self.manager.latest_checkpoint)
      if self.manager.latest_checkpoint:
          print("Restored from {}".format(self.manager.latest_checkpoint))
      else:
          print("Initializing from scratch.")

  def evaluate_loss(self, y_true, y_pred):
      f = tf.keras.losses.MeanSquaredError()
      l = []
      for i in range(y_true.shape[0]):
        l1 = f(y_true[i,:], y_pred[i,:])
        l2 = f(y_true[i,:], - y_pred[i,:])
        l.append(np.min([l1,l2]))
      return np.mean(l)

  def loss_val(self, x_val, y_val):
      '''
      Computes the validation loss 
      Args:
        x_val(tensor): batch of validation sample
        y_val (tensor): labels for validation
      Returns:
         val_loss(tensor): validation loss
      '''
      pred_val = self.net(x_val, False)
      val_loss = self.loss(pred_val, y_val)
      return val_loss

  def early_stop(self, epoch, val_loss, stop):
      '''
      Assesses if we have to stop training
      Args:
         epoch (int): current epoch
         val_loss (tensor): current validation loss
         stop (bool): early stop parameter
      Returns:
         stop(bool): True if the models stops training, false if it continues training
      '''
      #Store best validation loss
      if epoch == 0:
          self.best_loss = val_loss
      else:
          if val_loss < self.best_loss:
              self.stopping_step = 0
              self.best_loss = val_loss
          else:
              #If the validation loss does not decrease, we increase the number of stopping steps
              self.stopping_step += 1
      #If such number reaches the maximum, we stop training
      if self.stopping_step == self.early_stopping:
          stop = True
          print('Early stopping was triggered ')
      return stop

    # Optimization process. 
  @tf.function()
  def run_optimization(self,x, y):
      '''
      Performs one step of the learning process. It calculates the loss function and
      appies backpropagation algorithm to update the weights.

      Args:
        x (tensor): Samples of training data used to train the model
        y (tensor): Labels for training data
      
      Returns:
        -
      '''
      # Wrap computation inside a GradientTape for automatic differentiation.
      with tf.GradientTape() as g:
          # Forward pass.
          pred = self.net(x)
          # Compute loss.
          loss = self.loss(pred, y)
          
      # Variables to update, i.e. trainable variables.
      trainable_variables = self.net.trainable_variables

      # Compute gradients.
      gradients = g.gradient(loss, trainable_variables)
      
      # Update W and b following gradients.
      self.optimizer.apply_gradients(zip(gradients, trainable_variables))
      return loss

  #@tf.function
  def fit(self, X_train,y_train, X_test,y_test, save=True):
    '''
    Main fit function 

    Args:
      X_train (numpy array): Processed training data
      y_train (numpy array): Labels training data
      X_test (numpy array): Processed test data
      y_test (numpy array): Labels test data
      save (bool): If true, we save the weights at the end of the training
    Returns:
      -
    '''
    # Create train and test datasets
    # Use tf.data API to shuffle and batch data.
    train_data = tf.data.Dataset.from_tensor_slices((X_train, y_train))
    train_data = train_data.repeat().shuffle(5000).batch(self.batch_size).prefetch(1)
    

    test_data = tf.data.Dataset.from_tensor_slices((X_test, y_test))
    test_data = test_data.shuffle(buffer_size=1024).batch(self.batch_size) 

    loss_batch = []
    val_loss_batch = []

    stop = False
    epoch = 0
    
    # Run training for the given number of steps (and while not early stopping).
    while epoch < self.training_iters and stop == False:
        for step, (batch_x_train, batch_y_train) in enumerate(train_data.take(self.training_iters), 1):
            #Apply backpropagation algorithm
            loss = self.run_optimization(batch_x_train, batch_y_train)
            loss_batch.append(loss.numpy())

        for (test_x, test_y) in test_data:
            #Compute validation loss
            val_loss = self.loss_val(test_x, test_y)
            val_loss_batch.append(val_loss.numpy())
        
        stop = self.early_stop(epoch, val_loss, stop)
        epoch += 1

        #Display the result
        if epoch % self.display_step == 0:
          print('Epoch: ', epoch, "Validation loss: ", val_loss.numpy(), "Loss: ", loss.numpy())
    
    #Save the weights
    if save:
      save_path = self.manager.save()
      print("Saved checkpoint for step {}".format(save_path))    

In [77]:
#@title Split train and test sets { form-width: "30%" }
from sklearn.model_selection import train_test_split

# Split train and test 
idx_train, idx_test, wave_train, wave_test = train_test_split(np.arange(V.shape[0]),
                                                              waves, test_size=0.33, random_state=123)
V_train = V[idx_train,:]
V_test = V[idx_test,:]

alpha_train = alpha[idx_train]
alpha_test = alpha[idx_test]


In [78]:
#@title Parameters to tran the neural network { output-height: 10 }

#@markdown ---
#@markdown ### Enter the training parameters:
learning_rate = 0.0005#@param {type:"number"}
n_points = 200#@param{type:"integer"}
training_iters = 1000 #@param {type:"integer"}
batch_size = 64#@param {type:"integer"}
display_step = 10#@param {type:"integer"}
filepath ="./trained_models/random_potentials/1D/n0/tf_ckpts/" #@param {type:"string"}
#@markdown ---



In [79]:
#@title Run to load model { form-width: "30%" }
fc_model = FC_Model(input_size=n_points)
train = Training(fc_model,learning_rate, training_iters, batch_size, display_step,
                 filepath=filepath,early_stopping=100, restore=True)

Restored from ./trained_models/random_potentials/1D/n0/tf_ckpts/ckpt-1


In [ ]:
#@title Run to train model { form-width: "30%" }
#@markdown **Note: This cell may take hours to run, you may consider loading the model instead**
fc_model = FC_Model(input_size=n_points)

train = Training(fc_model,learning_rate, training_iters, batch_size, display_step, 
                 filepath=filepath,early_stopping=100, restore=False)

train.fit(V_train, wave_train, V_test, wave_test)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch:  10 Validation loss:  7.086477853590623e-05 Loss:  3.2300893508363515e-05
Epoch:  20 Validation loss:  1.2797223462257534e-05 Loss:  1.5235496903187595e-05
Epoch:  30 Validation loss:  3.930790626327507e-06 Loss:  3.619705012170016e-06
Epoch:  40 Validation loss:  3.827545242529595e-06 Loss:  5.185217560210731e-06
Epoch:  50 Validation loss:  3.2233790534519358e-06 Loss:  4.339248789619887e-06
Epoch:  60 Validation loss:  3.6811040899920044e-06 Loss:  1.4895451840857277e-06
Epoch:  70 Validation 

In [80]:
#@title Make predictions
pred = fc_model(V_test)
print("Test MSE: %f" % train.loss(pred, wave_test))

Test MSE: 0.000004


## Plot the results

We plot the prediction and the real values of the wave function.

In [81]:
#@title
idx = 1
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(
    x=x,
    y=pred.numpy()[idx,:],
    mode="lines",
    name = "prediction phi"),
    secondary_y=False
    )

fig.add_trace(go.Scatter(
    x=x,
    y=wave_test[idx,:],
    mode="lines",
    name = "real phi"),
    secondary_y=False
    )

fig.add_trace(go.Scatter(
    x=x,
    y=V_test[idx,:],
    mode="lines",
    name = "Potential"),
    secondary_y=True
    )

fig.update_layout(
    title="Random potential prediction",
    xaxis_title="x")

fig.show()

In [83]:
#@title Code to calculate the empirical energy
#@markdown Double click to show code

def empirical_energy1D( phi, potential, xmin=-8, xmax = 8,
                       n_points=200, hbar=1, m=1):
    # Normalize phi just in case
    h = (xmax - xmin)/n_points

    def energy(phi,potential,h):
      C = 1./np.sqrt(np.sum(phi*phi*h))
      phi = C*phi
      # We first calculate the second derivative of phi
      phir = np.concatenate(( phi[1:], np.zeros(1)), axis=0) # We add 0 at the extrema. It makes sense because phi(x)->0 at x->+-inf
      phil = np.concatenate(( np.zeros(1), phi[:-1]), axis=0)
      
      deriv = (phir - 2*phi + phil)/(h*h)
      return np.sum((-hbar*hbar/(2*m)*phi*deriv + potential*(phi*phi))*h)

    E = np.array([energy(phi[i,:], potential[i,:], h) for i in range(phi.shape[0])])   
    return E


In [84]:
E = empirical_energy1D(wave_test, V_test)
E_emp = empirical_energy1D(pred, V_test)
print('MSE(E): ', np.mean((E - E_emp)**2))

MSE(E):  1.544340179994611e-08


In [23]:
#@title Real and predicted mean energies
import plotly
from plotly.graph_objs import graph_objs as go
from plotly.subplots import make_subplots
from plotly.offline import iplot

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=E,
    y=E_emp,
    mode="markers", name = "empirical"))

fig.add_trace(go.Scatter(
    x=E,
    y=E,
    mode="lines", name = "theorerical"))

fig.update_layout(
    title="Random potential energies",
    xaxis_title="Theoretical",
    yaxis_title="Empirical")

fig.show()

## Test with Harmonic Oscillator

In [85]:
#@title Class to generate data of the Harmonic Oscillator
#@markdown Double click to see the code

import numpy as np
from scipy.special import eval_hermite
import math 

class Harmonic_Oscillator:

  def __init__(self, omega_min = 0.001, omega_max = 1, x0_min = -0.5, x0_max = 0.5,
               hbar = 1, m = 1, x_range = 0.8, n_points = 200):
    self.omega_min = omega_min # omega ~ U(omega_min, omega_max)
    self.omega_max = omega_max
    self.x0_min = x0_min
    self.x0_max = x0_max
    self.hbar = hbar
    self.m = m
    self.find_xrange(x_range) # x in [-3 smax, 3smax]
    self.n_points = n_points # Number of points of the grid 
    self.omega=None
    self.x0 = None
  
  def find_xrange(self, x_range):
    '''
    Find range of x values x \in [-x_range * sigma_max, + x_range * sigma_max]
    '''
    smax = np.sqrt(self.hbar/(self.m*self.omega_min)) # Find sigma_max
    self.xmin = -x_range*smax
    self.xmax = x_range*smax

  def generate_omega(self, N):
    if N==None:
      N = 100
    self.omega = np.random.uniform(self.omega_min, self.omega_max, N).reshape(-1,1)
    self.x0 = np.random.uniform(self.x0_min, self.x0_max, N).reshape(-1,1)

  def generate_data(self, N=None, n = 0, new_omega=True):
    '''
    Generates N random data points from the energetic level n
    '''
    if new_omega or self.omega.any()==None or self.x0.any()==None:
      self.generate_omega(N)

    x = np.arange(self.xmin, self.xmax, (self.xmax - self.xmin)/self.n_points)
    sigma_inv = np.sqrt(self.m*self.omega/self.hbar).reshape(-1,1)
    ones = np.repeat(1, N)
    x_mat = np.tensordot(x, ones, axes=0).T
    all_x = (x_mat - self.x0)*sigma_inv # It is a matrix of dim (num_omega x num_x_points), 
    # where each row has the values of x times sqrt(m*omega/hbar. In each row we change the value of omega
    herm = eval_hermite(n, all_x) # H_n(x/sigma)
    exp = np.exp(- all_x**2/2) # Exponential term

    phi_n = exp*herm

    h = (self.xmax - self.xmin)/self.n_points
    C = 1./np.sqrt(np.sum(phi_n*phi_n*h, axis = 1)) #1/np.sqrt(2**n * math.factorial(int(n))) * np.sqrt(sigma_inv)# Normalization constant
    C = C.reshape(-1,1)

    phi_n = C*phi_n#exp*herm

    potential = (x_mat-self.x0)**2 * 1/2*self.m*self.omega**2

    return phi_n, x, self.omega, self.x0, potential

  def get_energy(self, n, omega=np.array([None])):
    if omega.any()==None:
      omega = self.omega
    E = self.hbar*omega*(n+1/2)
    return E.flatten()


  def empirical_energy(self, phi, potential):
    # Normalize phi just in case
    h = (self.xmax - self.xmin)/self.n_points
    #C = 1./np.sqrt(np.sum(phi*phi*h))
    #phi = C*phi

    # We first calculate the second derivative of phi
    phir = np.concatenate(( phi[1:], np.zeros(1)), axis=0) # We add 0 at the extrema. It makes sense because phi(x)->0 at x->+-inf
    phil = np.concatenate(( np.zeros(1), phi[:-1]), axis=0)
    deriv = (phir - 2*phi + phil)/(h*h)

    # Now we calculate the mean energy
    energy = np.sum((-self.hbar*self.hbar/(2*self.m)*phi*deriv + potential*(phi*phi))*h)
    return energy



In [86]:
#@title Generate H.O data { form-width: "30%" }
ho = Harmonic_Oscillator(n_points =n_points, omega_min=0.2, omega_max = 1.0, x0_min = -0.005, x0_max = 0.005)
ho.xmax = 8
ho.xmin=-8

phi_0, x, omega, x0, potential = ho.generate_data(1000,0)

In [87]:
#@title Predict wavefunctions
pred = fc_model(potential)
print("Test MSE: %f" % train.loss(pred, phi_0))

Test MSE: 0.000025


In [ ]:
#@title Real and predicted wavefunctions
idx = 1
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(
    x=x,
    y=pred.numpy()[idx,:],
    mode="lines",
    name = "prediction phi"),
    secondary_y=False
    )

fig.add_trace(go.Scatter(
    x=x,
    y=phi_0[idx,:],
    mode="lines",
    name = "real phi"),
    secondary_y=False
    )

fig.add_trace(go.Scatter(
    x=x,
    y=potential[idx,:],
    mode="lines",
    name = "Potential"),
    secondary_y=True
    )

fig.update_layout(
    title="Harmonic oscillator prediction",
    xaxis_title="x")

fig.show()

In [89]:
E = empirical_energy1D(phi_0, potential)
E_emp = empirical_energy1D(pred, potential)

print('MSE(E): ', np.mean((E - E_emp)**2))

MSE(E):  3.4382103420820473e-07


In [ ]:
#@title Real and predicted mean energies
import plotly
from plotly.graph_objs import graph_objs as go
from plotly.subplots import make_subplots
from plotly.offline import iplot

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=E,
    y=E_emp,
    mode="markers", name = "empirical"))

fig.add_trace(go.Scatter(
    x=E,
    y=E,
    mode="lines", name = "theorerical"))

fig.update_layout(
    title="Harmonic oscillator energies",
    xaxis_title="Theoretical",
    yaxis_title="Empirical")

fig.show()

## Test with Morse potential

In [30]:
#@title Class to generate Morse potential and wavefunctions
#@markdown Double click to see the code
import numpy as np
from scipy.special import eval_genlaguerre
from scipy.special import gamma, factorial

class Morse:

  def __init__(self, a_min = 0.05, a_max = 0.1, hbar = 1, De_min= 0.1,
               De_max = 0.5, xe_min =-5, xe_max = 5, m=1,
               x_min = -30, x_max=30, n_points = 200):
    self.a_min = a_min # a ~ U(a_min, a_max)
    self.a_max = a_max
    self.m = m
    self.xe_min = xe_min
    self.xe_max = xe_max
    self.hbar = hbar
    self.De_min = De_min
    self.De_max = De_max
    self.xmin = x_min
    self.xmax = x_max
    self.n_points = n_points # Number of points of the grid 
    self.a=None
    self.xe = None
    self.De = None
  

  def generate_a(self, N):
    if N==None:
      N = 100
    self.a = np.random.uniform(self.a_min, self.a_max, N).reshape(-1,1)
    self.xe = np.random.uniform(self.xe_min, self.xe_max, N).reshape(-1,1)
    self.De = np.random.uniform(self.De_min, self.De_max, N).reshape(-1,1)

  def generate_data(self, N=None, n = 0, new_a=True):
    '''
    Generates N random data points from the energetic level n
    '''
    if new_a or self.a.any()==None or self.xe.any()==None or self.De.any()==None:
      self.generate_a(N)

    x = np.arange(self.xmin, self.xmax, (self.xmax - self.xmin)/self.n_points) # Grid of x values
    lamb = np.sqrt(2*self.m*self.De)/(self.a*self.hbar)
    lamb = lamb.reshape(-1,1)

    ones = np.repeat(1, N)
    x_mat = np.tensordot(x, ones, axes=0).T

    y = (x_mat-self.xe)*self.a
    z = 2*lamb*np.exp(-y)

    exp = np.exp(-1/2*z) #exponential term
    pot = z**(lamb -n -1/2) # Potential term

    laguerre = eval_genlaguerre(n, 2*lamb - 2*n -1, z)

    phi_n = pot*exp*laguerre

    h = (self.xmax - self.xmin)/self.n_points
    C = 1./np.sqrt(np.sum(phi_n*phi_n*h, axis = 1)) #np.sqrt(factorial(int(n)) * (2*lamb - 2*n - 1)/gamma(2*lamb - n)) # Normalization constant
    C = C.reshape(-1,1)

    phi_n = C*phi_n

    potential = self.De*(np.exp(-2*y) - 2*np.exp(-y))

    return phi_n, x, self.a, self.De, self.xe, potential


  def get_energy(self, n, a = None, De = None):
    if a.any()==None:
      a = self.a
    if De.any()==None:
      De = self.De
    lamb = np.sqrt(2*self.m*De)/(a*self.hbar)
    E = - self.hbar*self.hbar*a*a/(2*self.m)*(lamb - n-1/2)**2
    return E.flatten()


  def empirical_energy(self, phi, potential):
    # Normalize phi just in case
    h = (self.xmax - self.xmin)/self.n_points
    C = 1./np.sqrt(np.sum(phi*phi*h))
    phi = C*phi
    # We first calculate the second derivative of phi
    phir = np.concatenate(( phi[1:], np.zeros(1)), axis=0) # We add 0 at the extrema. It makes sense because phi(x)->0 at x->+-inf
    phil = np.concatenate(( np.zeros(1), phi[:-1]), axis=0)
    
    deriv = (phir - 2*phi + phil)/(h*h)

    # Now we calculate the mean energy
    energy = np.sum((-self.hbar*self.hbar/(2*self.m)*phi*deriv + potential*(phi*phi))*h)
    return energy






In [90]:
#@title Generate Morse Data { form-width: "30%" }
morse = Morse(n_points = n_points,  x_min = -8, x_max=8, xe_min =-0.5,
              xe_max = 0.5, De_max = 7, De_min =3)

phi_0, x, a, De, xe, potential = morse.generate_data(100,0, True) 

In [91]:
#@title Predict wavefunctions
pred = fc_model(potential)
print("Test MSE: %f" % train.loss(pred, phi_0))

Test MSE: 0.000135


In [ ]:
#@title Real and predicted wavefunctions
idx = 1
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(
    x=x,
    y=pred.numpy()[idx,:],
    mode="lines",
    name = "prediction phi"),
    secondary_y=False
    )

fig.add_trace(go.Scatter(
    x=x,
    y=phi_0[idx,:],
    mode="lines",
    name = "real phi"),
    secondary_y=False
    )

fig.add_trace(go.Scatter(
    x=x,
    y=potential[idx,:],
    mode="lines",
    name = "Potential"),
    secondary_y=True
    )

fig.update_layout(
    title="Morse prediction",
    xaxis_title="x")

fig.show()

In [92]:
E = empirical_energy1D(phi_0, potential)
E_emp = empirical_energy1D(pred, potential)

print('MSE(E): ', np.mean((E - E_emp)**2))

MSE(E):  1.6992429143981118e-06


In [ ]:
#@title Real and predicted mean energies
import plotly
from plotly.graph_objs import graph_objs as go
from plotly.subplots import make_subplots
from plotly.offline import iplot

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=E,
    y=E_emp,
    mode="markers", name = "empirical"))

fig.add_trace(go.Scatter(
    x=E,
    y=E,
    mode="lines", name = "theorerical"))

fig.update_layout(
    title="Morse energies",
    xaxis_title="Theoretical",
    yaxis_title="Empirical")

fig.show()

# Excited state potentials

Now that we have generated the code to approximate the eigenstates of a random potential, we are going to generate random potentials of the form:

$$
V(x) = \alpha_0 + \alpha_1 x + \alpha_2 x^2 + \alpha_3 x^3 + \alpha_4 x^4
$$

To ensure that the eigenstates have discrete energies (and thus are physical states), we will impose some properties on the coefficients, so that the even terms ($x^2$ and $x^4$) dominate over the odd term ($x^3$). Also, we allow the potential to be negative and non-centered by including negative values of $\alpha_0$ and $\alpha_1$. Finally, we will use small values of the coefficients so that the potential does not achieve very high values, which can lead to numerical unstability.

We will test the method for high-enery excited states.


In [ ]:
#@title Generate data

#@markdown ---
#@markdown ### Enter the data parameters:
xmin = -20#@param {type:"number"}
xmax = 20#@param {type:"number"}
N=100#@param {type:"slider", min:80, max:120, step:1}
n_state=10#@param {type:"slider", min:0, max:10, step:1}
n_points = 200#@param {type:"integer"}
n_samples=100#@param {type:"integer"}
#@markdown ---



# We generate the values of alpha
alpha_min = np.array([-900,-130,0.3,-0.1,0])/200
alpha_max = np.array([-300,5,1.0,0.1,0.2])/200
k = alpha_min.shape[0]

r_alpha = np.random.random((int(n_samples*0.6), k)) # Values between 0 and 1
alpha1 = r_alpha*(alpha_max - alpha_min)+ alpha_min # random alpha

alpha_min2 = np.array([-0.5,-0.1,0.2,0,0])/20
alpha_max2 = np.array([0.5,0.1,1.0,0,0])/20

r_alpha = np.random.random((int(n_samples*0.2), k)) # Values between 0 and 1
alpha2 = r_alpha*(alpha_max2 - alpha_min2)+ alpha_min2 # random alpha

alpha_min3 = np.array([-300,-10,0.1,-0.1,0])/100
alpha_max3 = np.array([-20,10,1.0,0.1,0])/100

r_alpha = np.random.random((int(n_samples*0.2), k)) # Values between 0 and 1
alpha3 = r_alpha*(alpha_max3 - alpha_min3)+ alpha_min3 # random alpha

alpha = np.concatenate((alpha1, alpha2, alpha3))

data_gen = eigen_state_potential(alpha_min, alpha_max, N)

## Convergence of the method

In order to choose the value of $N$, we will generate the ground-state energies for different vales of $N$ and compute the difference. When the value of the energy stabilizes up to certain precision $\gamma$, we will fix the value of $N$. We will evaluate the performance for 500 different samples.

**Note: The following cells may take hours to run**

In [ ]:
gamma = 0.005
Ns = np.arange(50,101)
n_state = 10
Es = np.zeros(( Ns.shape[0], n_samples))

for i in range(Ns.shape[0]):
  print("\nState: ", i+50)
  data_gen = eigen_state_potential(alpha_min, alpha_max, Ns[i])
  E, a, _ = data_gen.generate_data(n_samples, alpha, n_state)
  Es[i,:] = E

In [ ]:
#@title Energies as a function of N
import plotly
from plotly.graph_objs import graph_objs as go
from plotly.subplots import make_subplots
from plotly.offline import iplot


fig = go.Figure()
fig.add_trace(go.Scatter(
    x=Ns,
    y=Es[:,0],
    mode="lines",
    name = "Energy evolution")
    )

fig.add_trace(go.Scatter(
    x=Ns,
    y=np.repeat(Es[-1,0], Ns.shape[0]),
    mode="lines",
    name = "Final energy")
    )

fig.update_layout(
    title="Energies as a function of N",
    xaxis_title="N")

fig.show()

We see that the energy stabilizes fast. Now, for $\gamma =0.001$, let's see which is the $N$ needed on average to ensure convergence.

In [ ]:
#@title Evolution of difference of energy
gamma = 0.005
E_dif = np.abs(Es[:-1,:] - Es[1:,:])

E_dif_mean = np.mean(E_dif, axis=1)

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=Ns[1:],
    y=E_dif_mean,
    mode="lines",
    name = "Difference in energy")
    )
fig.add_trace(go.Scatter(
    x=Ns[:-1:],
    y=np.repeat(gamma, (Ns[1:]).shape[0]),
    mode="lines",
    name = "Gamma")
    )

We see that for $N=80$ the difference in the ground energy is below $\gamma$. Thus, we select this value to generate the wavefunctions. 

In [ ]:
#@title Generate data
#@markdown **Note: this cell may take hours to run. 
#@markdown You can load the data instead**

#@markdown ---
#@markdown ### Enter the data parameters:
xmin = -20#@param {type:"number"}
xmax = 20#@param {type:"number"}
N=90#@param {type:"slider", min:50, max:120, step:1}
n_state=10#@param {type:"slider", min:0, max:20, step:1}
n_points = 200#@param {type:"integer"}
n_samples=5000#@param {type:"integer"}
#@markdown ---


# We generate the values of alpha
alpha_min = np.array([-900,-130,0.3,-0.1,0])/200
alpha_max = np.array([-300,5,1.0,0.1,0.2])/200

r_alpha = np.random.random((int(n_samples*0.6), k)) # Values between 0 and 1
alpha1 = r_alpha*(alpha_max - alpha_min)+ alpha_min # random alpha

alpha_min2 = np.array([-0.5,-0.01,0.2,0,0])
alpha_max2 = np.array([0.5,0.01,1.0,0,0])

r_alpha = np.random.random((int(n_samples*0.2), k)) # Values between 0 and 1
alpha2 = r_alpha*(alpha_max2 - alpha_min2)+ alpha_min2 # random alpha

alpha_min3 = np.array([-300,-10,0.1,-0.1,0])/100
alpha_max3 = np.array([-20,10,1.0,0.1,0])/100

r_alpha = np.random.random((int(n_samples*0.2), k)) # Values between 0 and 1
alpha3 = r_alpha*(alpha_max3 - alpha_min3)+ alpha_min3 # random alpha

alpha = np.concatenate((alpha1, alpha2, alpha3))

data_gen = eigen_state_potential(alpha_min, alpha_max, N)

# Generate the energies, wavefunctions and potentials
E, a, alpha = data_gen.generate_data(n_samples, alpha, n_state)
waves, x,phis = data_gen.final_wavefunction( xmin, xmax, n_points, a)
V, _ = data_gen.evaluate_potential( xmin, xmax, n_points, alpha)
idx=-1

In [ ]:
#@title Code to save the generated data
#@markdown Double click to see the code \\
#@markdown **Note: Only run this cell if you have generated new data and want 
#@markdown  to overwrite the previously saved one. Notice that this could alter 
#@markdown the results of the notebook.**
# Save data
with open("./training_data/random_potentials/1D/n10/V.npy", 'wb') as f:
  np.save(f, V)
with open("./training_data/random_potentials/1D/n10/waves.npy", 'wb') as f:
  np.save(f, waves)
with open("./training_data/random_potentials/1D/n10/x.npy", 'wb') as f:
  np.save(f, x)
with open("./training_data/random_potentials/1D/n10/alpha.npy", 'wb') as f:
  np.save(f, alpha)
with open("./training_data/random_potentials/1D/n10/a.npy", 'wb') as f:
  np.save(f, a)

In [93]:
#@title Code to load the data { vertical-output: true }
#@markdown Double click to see the code \\
#@markdown  **Note: Run this cell to load the saved data**
# Load data
with open("./training_data/random_potentials/1D/n10/V.npy", 'rb') as f:
  V = np.load(f)
with open("./training_data/random_potentials/1D/n10/waves.npy", 'rb') as f:
  waves = np.load(f)
with open("./training_data/random_potentials/1D/n10/x.npy", 'rb') as f:
  x = np.load(f)
with open("./training_data/random_potentials/1D/n10/alpha.npy", 'rb') as f:
  alpha = np.load(f)
with open("./training_data/random_potentials/1D/n10/a.npy", 'rb') as f:
  a = np.load(f)

In [94]:
#@title Example of wavefunctions
import plotly
from plotly.graph_objs import graph_objs as go
from plotly.subplots import make_subplots
from plotly.offline import iplot

idx =1

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(
    x=x,
    y=waves[idx,:],
    mode="lines",
    name = "solution")
    )

fig.add_trace(go.Scatter(
    x=x,
    y=V[idx,:],
    mode="lines",
    name = "potential"),
    secondary_y=True
    )

fig.update_layout(
    title="Approximate wavefunctions",
    xaxis_title="x")

fig.show()



In [95]:
#@title Split training and test data { form-width: "30%" }
from sklearn.model_selection import train_test_split

# Split train and test 
V_train, V_test, wave_train, wave_test = train_test_split(V, waves, test_size=0.33, random_state=42)



In [96]:
#@title Parameters to tran the neural network { output-height: 10 }
#@markdown You can also include Markdown in forms.

#@markdown ---
#@markdown ### Enter the training parameters:
n_points = 200#@param {type:"integer"}
learning_rate = 0.0001#@param {type:"number"}
training_iters = 1000 #@param {type:"integer"}
batch_size = 64#@param {type:"integer"}
display_step = 10#@param {type:"integer"}
filepath = "./trained_models/random_potentials/1D/n10/tf_ckpts/" #@param {type:"string"}
#@markdown ---

#@markdown ---
#@markdown ### Enter the network parameters:
n1 = 256#@param {type:"integer"}
n2= 128#@param {type:"integer"}
n3=128#@param {type:"integer"}
n4=256#@param {type:"integer"}
drop=0.2#@param {type:"number"}
#@markdown ---




In [ ]:
#@title Run to load model { form-width: "30%" }
fc_model = FC_Model(input_size=n_points,n1=n1, n2=n2, n3=n3, n4=n4, drop=drop)
train = Training(fc_model,learning_rate, training_iters, batch_size, display_step,
                 filepath=filepath,early_stopping=200, restore=True)


In [ ]:
#@title Run to train model { form-width: "30%" }
#@markdown **Note: This cell may take hours to run, you may consider loading the model instead**
fc_model = FC_Model(input_size=n_points,n1=n1, n2=n2, n3=n3, n4=n4, drop=drop)

train = Training(fc_model,learning_rate, training_iters, batch_size, 
                 display_step, filepath=filepath,early_stopping=200, restore=False)

train.fit(V_train, wave_train, V_test, wave_test, save=False)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch:  10 Validation loss:  0.004400508478283882 Loss:  0.002998128067702055
Epoch:  20 Validation loss:  0.0008483840501867235 Loss:  0.0006320091197267175
Epoch:  30 Validation loss:  0.0002512630308046937 Loss:  0.00024103716714307666
Epoch:  40 Validation loss:  0.0003794087388087064 Loss:  0.00025719741825014353
Epoch:  50 Validation loss:  0.00011859538062708452 Loss:  8.492523920722306e-05
Epoch:  60 Validation loss:  0.00031676021171733737 Loss:  0.00018684883252717555
Epoch:  70 Validation los

In [152]:
#@title Make predictions
pred = fc_model(V_test)
print("Test MSE: %f" % train.evaluate_loss(pred, wave_test))

Test MSE: 0.000009


## Plot the results

We plot the prediction and the real values of the wave function.

In [ ]:
#@title
idx +=1
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(
    x=x,
    y=pred.numpy()[idx,:],
    mode="lines",
    name = "prediction phi"),
    secondary_y=False
    )

fig.add_trace(go.Scatter(
    x=x,
    y=wave_test[idx,:],
    mode="lines",
    name = "real phi"),
    secondary_y=False
    )

fig.add_trace(go.Scatter(
    x=x,
    y=V_test[idx,:],
    mode="lines",
    name = "Potential"),
    secondary_y=True
    )

fig.update_layout(
    title="Random potential prediction",
    xaxis_title="x")

fig.show()

In [100]:
E = empirical_energy1D(wave_test, V_test, xmin = -20, xmax =20, n_points = 200)
E_emp = empirical_energy1D(pred, V_test, xmin = -20, xmax =20, n_points = 200)

print('MSE(E) = ', np.mean((E-E_emp)**2))

MSE(E) =  6.458685102182984e-07


In [47]:
#@title Real and predicted mean energies
import plotly
from plotly.graph_objs import graph_objs as go
from plotly.subplots import make_subplots
from plotly.offline import iplot

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=E,
    y=E_emp,
    mode="markers", name = "empirical"))

fig.add_trace(go.Scatter(
    x=E,
    y=E,
    mode="lines", name = "theorerical"))

fig.update_layout(
    title="Random potential energies",
    xaxis_title="Theoretical",
    yaxis_title="Empirical")

fig.show()

## Test with Harmonic Oscillator

In [150]:
#@title Generate H.O data { form-width: "30%" }
ho = Harmonic_Oscillator(n_points =n_points, omega_min=0.65, omega_max = 0.9, 
                         x0_min = -0.005, x0_max = 0.005)
ho.xmax = 20
ho.xmin=-20
n_state=10

phi_0, x, omega, x0, potential = ho.generate_data(1000,n_state)

In [151]:
#@title Predict wavefunctions
pred = fc_model(potential)
print("Test MSE: %f" % train.evaluate_loss(pred, phi_0))
idx=0

Test MSE: 0.000003


In [50]:
#@title Real and predicted wavefunctions
idx += 1
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(
    x=x,
    y=pred.numpy()[idx,:],
    mode="lines",
    name = "prediction phi"),
    secondary_y=False
    )

fig.add_trace(go.Scatter(
    x=x,
    y=phi_0[idx,:],
    mode="lines",
    name = "real phi"),
    secondary_y=False
    )

fig.add_trace(go.Scatter(
    x=x,
    y=potential[idx,:],
    mode="lines",
    name = "Potential"),
    secondary_y=True
    )

fig.update_layout(
    title="Harmonic oscillator prediction",
    xaxis_title="x")

fig.show()

In [103]:
E = empirical_energy1D(phi_0, potential,xmin=-20, xmax = 20, n_points=200)
E_emp = empirical_energy1D(pred, potential,xmin=-20, xmax = 20, n_points=200)

print('MSE(E) = ', np.mean((E-E_emp)**2))

MSE(E) =  2.206082232468438e-07


In [52]:
#@title Real and predicted mean energies
import plotly
from plotly.graph_objs import graph_objs as go
from plotly.subplots import make_subplots
from plotly.offline import iplot

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=E,
    y=E_emp,
    mode="markers", name = "empirical"))

fig.add_trace(go.Scatter(
    x=E,
    y=E,
    mode="lines", name = "theorerical"))

fig.update_layout(
    title="Harmonic oscillator energies",
    xaxis_title="Theoretical",
    yaxis_title="Empirical")

fig.show()

## Test with Morse potential

In [143]:
#@title Generate Morse Data { form-width: "30%" }
morse = Morse(n_points = n_points,  x_min = -20, x_max=20, xe_min =-5,
              a_min = 0.05, a_max = 0.09,
              xe_max = -4, De_max = 7, De_min =3)

phi_0, x, a, De, xe, potential = morse.generate_data(1000,n_state, True) 
idx=0

In [149]:
#@title Predict wavefunctions
pred = fc_model(potential)
print("Test MSE= {}".format(train.evaluate_loss(pred, phi_0)))

Test MSE= 0.0017051184305804782


In [107]:
#@title Real and predicted wavefunctions
import plotly
from plotly.graph_objs import graph_objs as go
from plotly.subplots import make_subplots
from plotly.offline import iplot

idx += 1
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(
    x=x,
    y=pred.numpy()[idx,:],
    mode="lines",
    name = "prediction phi"),
    secondary_y=False
    )

fig.add_trace(go.Scatter(
    x=x,
    y=phi_0[idx,:],
    mode="lines",
    name = "real phi"),
    secondary_y=False
    )

fig.add_trace(go.Scatter(
    x=x,
    y=potential[idx,:],
    mode="lines",
    name = "Potential"),
    secondary_y=True
    )

fig.update_layout(
    title="Morse prediction",
    xaxis_title="x")

fig.show()

In [133]:
E = empirical_energy1D(phi_0, potential,xmin=-20, xmax = 20, n_points=200)
E_emp = empirical_energy1D(pred, potential,xmin=-20, xmax = 20, n_points=200)

print('MSE(E) = ', np.mean((E-E_emp)**2))

MSE(E) =  0.000541540914121772


In [134]:
#@title Real and predicted mean energies
import plotly
from plotly.graph_objs import graph_objs as go
from plotly.subplots import make_subplots
from plotly.offline import iplot

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=E,
    y=E_emp,
    mode="markers", name = "empirical"))

fig.add_trace(go.Scatter(
    x=E,
    y=E,
    mode="lines", name = "theorerical"))

fig.update_layout(
    title="Morse energies",
    xaxis_title="Theoretical",
    yaxis_title="Empirical")

fig.show()